In [3]:
import os
os.chdir('../')
os.getcwd()

'/Users/dennis/repos/study/mt/mt_rdf'

In [4]:
import pandas as pd
from tqdm import tqdm
import src.helper.path_helper as ph

In [5]:
df_opts = pd.read_csv(ph.wdbench_query_path(ph.QueryType.opts), header=None)
df_opts.rename(columns={0: 'id', 1: 'query_parts'}, inplace=True)

df_c2rpqs = pd.read_csv(ph.wdbench_query_path(ph.QueryType.c2rpqs), header=None)
df_c2rpqs.rename(columns={0: 'id', 1: 'query_parts'}, inplace=True)

df_paths = pd.read_csv(ph.wdbench_query_path(ph.QueryType.paths), header=None)
df_paths.rename(columns={0: 'id', 1: 'query_parts'}, inplace=True)

df_single_bgps = pd.read_csv(ph.wdbench_query_path(ph.QueryType.single_bgps), header=None)
df_single_bgps.rename(columns={0: 'id', 1: 'query_parts'}, inplace=True)

df_multiple_bgps = pd.read_csv(ph.wdbench_query_path(ph.QueryType.multiple_bgps), header=None)
df_multiple_bgps.rename(columns={0: 'id', 1: 'query_parts'}, inplace=True)

In [6]:
query_stat_dict = {qtype.value: {} for qtype in ph.QueryType}

In [7]:
a = '?x1 <http://www.wikidata.org/prop/direct/P016> <http://www.wikidata.org/entity/Q13414980> . OPTIONAL { ?x1 <http://www.wikidata.org/prop/direct/P734> ?x2 . } OPTIONAL { ?x1 <http://www.wikidata.org/prop/direct/P569> ?x3 . } OPTIONAL { ?x1 <http://www.wikidata.org/prop/direct/P19> ?x4 . } OPTIONAL { ?x1 <http://www.wikidata.org/prop/direct/P570> ?x5 . } OPTIONAL { ?x1 <http://www.wikidata.org/prop/direct/P20> ?x6 . }'

In [8]:
import re
def get_query_stats(query_stat_dict, df, qtype):
    entity_pattern = r'<http://www\.wikidata\.org/entity/.+?>'
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        entities = re.findall(entity_pattern, row['query_parts'])
        query_stat_dict[qtype][row['id']] = entities
    
    return query_stat_dict

In [9]:
ph.QueryType.opts.value

'opts'

In [11]:
query_stat_dict = get_query_stats(query_stat_dict=query_stat_dict, df=df_opts, qtype=ph.QueryType.opts.value)
query_stat_dict = get_query_stats(query_stat_dict=query_stat_dict, df=df_c2rpqs, qtype=ph.QueryType.c2rpqs.value)
query_stat_dict = get_query_stats(query_stat_dict=query_stat_dict, df=df_multiple_bgps, qtype=ph.QueryType.multiple_bgps.value)
query_stat_dict = get_query_stats(query_stat_dict=query_stat_dict, df=df_paths, qtype=ph.QueryType.paths.value)
query_stat_dict = get_query_stats(query_stat_dict=query_stat_dict, df=df_single_bgps, qtype=ph.QueryType.single_bgps.value)

100%|██████████| 280/280 [00:00<00:00, 55120.86it/s]


In [12]:
dict_df = {'q_type': [], 'q_id': [], 'n_entities': [], 'entities': []}
for k, v in query_stat_dict.items():
    print(k)
    for id_key, entity_list in v.items():
        dict_df['q_type'].append(k)
        dict_df['q_id'].append(id_key)
        dict_df['n_entities'].append(len(entity_list))
        dict_df['entities'].append(entity_list)
    print(50*'_')

single_bgps
__________________________________________________
multiple_bgps
__________________________________________________
paths
__________________________________________________
c2rpqs
__________________________________________________
opts
__________________________________________________


In [13]:
stat_df = pd.DataFrame(data=dict_df)

In [14]:
stat_df.sort_values(by='n_entities', ascending=False)

,q_type,q_id,n_entities,entities
2531,opts,372,15,"[<http://www.wikidata.org/entity/Q5>, <http://..."
1746,c2rpqs,126,5,"[<http://www.wikidata.org/entity/Q16521>, <htt..."
1747,c2rpqs,127,5,"[<http://www.wikidata.org/entity/Q16521>, <htt..."
1748,c2rpqs,128,5,"[<http://www.wikidata.org/entity/Q16521>, <htt..."
693,multiple_bgps,414,4,"[<http://www.wikidata.org/entity/Q1025095>, <h..."
...,...,...,...,...
1606,paths,646,0,[]
1605,paths,645,0,[]
1082,paths,122,0,[]
1081,paths,121,0,[]


In [19]:
stat_df[(stat_df['n_entities'] >= 1) & (stat_df['q_type'] == 'paths')].sort_values(by='n_entities', ascending=False)

,q_type,q_id,n_entities,entities
1058,paths,98,2,"[<http://www.wikidata.org/entity/Q64>, <http:/..."
1065,paths,105,2,"[<http://www.wikidata.org/entity/Q80484>, <htt..."
1041,paths,81,2,"[<http://www.wikidata.org/entity/Q4176>, <http..."
1023,paths,63,2,"[<http://www.wikidata.org/entity/Q3044>, <http..."
1064,paths,104,2,"[<http://www.wikidata.org/entity/Q80484>, <htt..."
...,...,...,...,...
1189,paths,229,1,[<http://www.wikidata.org/entity/Q639669>]
1190,paths,230,1,[<http://www.wikidata.org/entity/Q82594>]
1191,paths,231,1,[<http://www.wikidata.org/entity/Q82955>]
1193,paths,233,1,[<http://www.wikidata.org/entity/Q33999>]


In [22]:
stat_df[(stat_df['q_type'] == 'paths') & (stat_df['q_id'] > 180) & (stat_df['entities'].str.contains('<http://www.wikidata.org/entity/Q3609>'))].sort_values(by='n_entities', ascending=False)

,q_type,q_id,n_entities,entities


In [27]:
pd.set_option('display.max_colwidth', None)

In [29]:
mask = stat_df.entities.apply(lambda x: '<http://www.wikidata.org/entity/Q3609>' in x)
df1 = stat_df[mask]
# df1['entities']
df1
               

,q_type,q_id,n_entities,entities
657,multiple_bgps,378,2,"[<http://www.wikidata.org/entity/Q31>, <http://www.wikidata.org/entity/Q3609>]"
716,multiple_bgps,437,2,"[<http://www.wikidata.org/entity/Q3609>, <http://www.wikidata.org/entity/Q64>]"
1609,paths,649,1,[<http://www.wikidata.org/entity/Q3609>]
2049,c2rpqs,429,2,"[<http://www.wikidata.org/entity/Q1194951>, <http://www.wikidata.org/entity/Q3609>]"


In [23]:
type(stat_df['entities'][0])

list

In [13]:
# todo: join with exec times

In [14]:
stat_df.groupby(['n_entities'])['n_entities'].count()

n_entities
0      252
1     1579
2      657
3      133
4       33
5        3
15       1
Name: n_entities, dtype: int64

In [15]:
df_paths_res = pd.read_csv('results/wdbench/results_paths.csv')

df_c2rpqs_res = pd.read_csv('results/wdbench/results_c2rpqs.csv')
df_multiple_bgps_res = pd.read_csv('results/wdbench/results_multiple_bgps.csv')
df_opts_res = pd.read_csv('results/wdbench/results_opts.csv')
df_single_bgps_res = pd.read_csv('results/wdbench/results_single_bgps.csv')

In [16]:
df_paths_res.columns

Index(['Unnamed: 0', 'query_id', 'exec_time'], dtype='object')

In [17]:
def prep_res_df(df, qtype):
    df.drop(['Unnamed: 0'], axis=1, inplace=True)
    df['q_type'] = qtype
    df.rename(columns={'query_id':'q_id'}, inplace=True)
    return df

In [18]:
df_paths_res = prep_res_df(df_paths_res, ph.QueryType.paths.value)
df_c2rpqs_res = prep_res_df(df_c2rpqs_res, ph.QueryType.c2rpqs.value)
df_multiple_bgps_res = prep_res_df(df_multiple_bgps_res, ph.QueryType.multiple_bgps.value)
df_opts_res = prep_res_df(df_opts_res, ph.QueryType.opts.value)
df_single_bgps_res = prep_res_df(df_single_bgps_res, ph.QueryType.single_bgps.value)

In [19]:
df_all_res = pd.concat([df_paths_res, df_c2rpqs_res, df_multiple_bgps_res, df_opts_res, df_single_bgps_res])

In [20]:
df_all_res

,q_id,exec_time,q_type
0,1,0.208867,paths
1,2,0.281636,paths
2,3,0.525953,paths
3,4,0.230193,paths
4,5,0.231809,paths
...,...,...,...
266,267,0.407957,single_bgps
267,268,0.147239,single_bgps
268,269,0.687138,single_bgps
269,270,0.154532,single_bgps


In [21]:
df_stats_q_time = pd.merge(stat_df, df_all_res, on=['q_type', 'q_id'])

In [22]:
df_stats_q_time[df_stats_q_time['exec_time'] > 3].sort_values(by='exec_time', ascending=False)

,q_type,q_id,n_entities,entities,exec_time
1833,c2rpqs,231,3,"[<http://www.wikidata.org/entity/Q377910>, <ht...",54.486320
1861,c2rpqs,259,2,"[<http://www.wikidata.org/entity/Q183>, <http:...",49.272325
1848,c2rpqs,246,2,"[<http://www.wikidata.org/entity/Q3305213>, <h...",43.716215
1743,c2rpqs,141,2,"[<http://www.wikidata.org/entity/Q82955>, <htt...",40.562598
926,multiple_bgps,656,0,[],35.459563
2060,c2rpqs,458,2,"[<http://www.wikidata.org/entity/Q702842>, <ht...",27.233602
2592,opts,459,1,[<http://www.wikidata.org/entity/Q5361461>],23.827203
2614,opts,481,1,[<http://www.wikidata.org/entity/Q42310659>],22.179871
596,multiple_bgps,326,2,"[<http://www.wikidata.org/entity/Q29>, <http:/...",18.393915
1807,c2rpqs,205,2,"[<http://www.wikidata.org/entity/Q39715>, <htt...",18.115517


In [24]:
df_stats_q_time[(df_stats_q_time['exec_time'] == 0) & (df_stats_q_time['n_entities'] == 2) & (df_stats_q_time['q_type'] == 'paths')].sort_values(by='n_entities', ascending=False)

,q_type,q_id,n_entities,entities,exec_time
1032,paths,81,2,"[<http://www.wikidata.org/entity/Q4176>, <http...",0.0
1055,paths,104,2,"[<http://www.wikidata.org/entity/Q80484>, <htt...",0.0
1056,paths,105,2,"[<http://www.wikidata.org/entity/Q80484>, <htt...",0.0


In [55]:
df_opts[df_opts['id'] == 486]

,id,query_parts
485,486,?x1 <http://www.wikidata.org/prop/direct/P495>...
